Move into `ControlNet-XS` directory to run

Note: I made these changes to make the code run:
1. In `requirements/pt2.txt`: `torchaudio==2.0.2`,`torchvision==0.15.2`
2. In `ldm/models/diffusion/ddpm.py`: `from pytorch_lightning.utilities.rank_zero import rank_zero_only`

In [1]:
import scripts.control_utils as cu
import torch
from PIL import Image

Info: `UmerDebugLogger` created. This is a logging class that will be deleted when the PR to integrate ControlNet-XS is done.


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


Downloading: "https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt" to /home/ControlNet-XS/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt



100%|██████████| 470M/470M [00:01<00:00, 284MB/s]  
/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [2]:
path_to_config = '../Heidelberg_model_configs/sd21_encD_canny_14m.yaml'
model = cu.create_model(path_to_config).to('cuda')

TwoStreamControlLDM: Running in eps-prediction mode
DiffusionWrapper has 865.91 M params.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla-xformers' with 512 in_channels
building MemoryEfficientAttnBlock with 512 in_channels...
[404 keys are missing from the model (hint processing and cross connections included)]
['CONTROL WEIGHTS LOADED']
Loaded model config from [../Heidelberg_model_configs/sd21_encD_canny_14m.yaml]


In [32]:
torch.set_printoptions(sci_mode=False, precision=3, linewidth=200)

def param_head(module, n=10, param='weight'):
    if 'TimestepEmbedSequential' in str(type(module)): module = module[0]
    return getattr(module, param).data.flatten()[:10]

def print_param_head(module, n=10, param='weight'):
    print(param_head(module=module, n=n, param=param))

In [33]:
print_param_head(model.control_model.control_model.input_blocks[1][0].in_layers[2])

tensor([ 0.055,  0.043,  0.043,  0.059,  0.023,  0.078,  0.047,  0.057,  0.037, -0.026], device='cuda:0')


In [34]:
print_param_head(model.control_model.control_model.input_blocks[1][0].in_layers[2], param='bias')

tensor([-0.025,  0.025,  0.072, -0.013, -0.131,  0.115, -0.010, -0.013,  0.028,  0.039], device='cuda:0')
